In [22]:
# Author: M. Riley Owens (GitHub: mrileyowens)

# This file determines how the Lyman-alpha flux
# in the MagE apertures changes due to seeing effects,
# as estimated from the narrowband Lyman-alpha
# images.

In [23]:
import os

from datetime import datetime, timezone

import sigfig

import numpy as np

from astropy.io import fits
from astropy.convolution import convolve_fft, Gaussian2DKernel

In [ ]:
def func():

    # Establish common directories
    home = os.getcwd()
    results = f'{home}/results'
    
    # Dictionary of the MagE slit aperture IDs, listing, from left to right: the FWHM of the seeing conditions 
    # in arcseconds, airmasses of each exposure, written as sec(z), and the exposure times of each exposure in seconds
    slits = {
        'M5' : [0.97, [1.756, 1.668, 1.602, 1.553, 1.531], [2700, 2700, 2700, 2700, 2700]],
        'M4' : [0.71, [1.529, 1.532, 1.549], [2400, 2400, 2400]],
        'M6' : [0.76, [1.530, 1.549, 1.581, 1.531, 1.552], [2700, 2400, 2400, 2700, 2700]],
        'M3' : [0.70, [1.775, 1.690, 1.627, 1.571, 1.542], [2400, 2400, 2400, 2400, 2400]],
        'M0' : [1.34, [1.540, 1.528, 1.536, 1.565, 1.656], [2700, 2700, 2700, 2700, 2700]],
        'M2' : [0.77, [1.584, 1.639, 1.592], [2700, 2700, 2700]],
        'M7' : [0.73, [1.563, 1.600, 1.665, 1.569, 1.610, 1.672], [2400, 2400, 2400, 2400, 2400, 2400]],
        'M8' : [0.70, [1.528, 1.537, 1.747, 1.529, 1.539, 1.757], [2400, 2400, 2400, 2400, 2400, 2400]],
        'M9' : [0.68, [1.738, 1.654, 1.592, 1.552, 1.531], [2700, 2700, 2700, 2700, 2700]]
    }

    # HST filter IDs used as continuum estimates for the narrowband
    # Lya maps
    filters = np.array(['F390W', 'F555W'], dtype=str)

    # File path to the mask of the two largest arcs of the Sunburst Arc
    # in the v5 HST data reduction WCS
    arc_mask_file = f'{results}/masks/arc_mask_v5.fits'

    # Get a cutout of the arc mask. This is necessary because we will
    # convolve the Lya maps later, which is computationally expensive for
    # a very large footprint.
    arc_mask = fits.getdata(arc_mask_file)[4000:5600, 4400:5700]

    # Make empty lists to append the results of the simulation to
    fluxes, n_fluxes, fluxes_convolved, n_fluxes_convolved, ratios, n_ratios = [], [], [], [], [], []

    # For each slit ID
    for i, slit_id in enumerate(slits):

        # Get the FWHM of the seeing conditions and the airmasses and exposure times of each exposure
        seeing, airmass, exposures = slits[slit_id][0], slits[slit_id][1], slits[slit_id][2]

        # File path to the slit mask
        slit_mask_file = f'{results}/masks/{slit_id}_mask_v5.fits'

        # Get a cutout of the mask
        slit_mask = fits.getdata(slit_mask_file)[4000:5600, 4400:5700]

        # For each filter used to estimate the continuum
        for j, filter in enumerate(filters):

            # File path to the corresponding Lya map
            file = f'{results}/lya_maps/Lya_cont_sub_{filter}.fits'

            # Get cutouts of the map and the uncertainty map
            lya = fits.open(file)[0].data[4000:5600, 4400:5700]
            n_lya = fits.open(file)[1].data[4000:5600, 4400:5700]
            
            # The standard deviation of the exposure time-weighted PSF in pixels
            stdv = ((seeing * (np.sum(np.multiply(airmass, exposures) / np.sum(exposures)))**0.6) / 2.355) / 0.03

            # A Gaussian kernel with the above standard deviation
            kernel = Gaussian2DKernel(x_stddev=stdv, y_stddev=stdv)

            # Convolve the Lya map with the Gaussian kernel representing the exposure time-weighted seeing conditions
            lya_convolved = convolve_fft(lya, kernel)
            
            # Sum the unconvolved Lya flux in the slit
            flux = np.sum(slit_mask * lya)
            fluxes.append(flux)

            # Calculate the propagated uncertainty of the unconvolved Lya flux in the slit
            n_flux = np.sqrt(np.sum((slit_mask * n_lya)**2))
            n_fluxes.append(n_flux)

            # Sum the convolved Lya flux in the slit
            flux_convolved = np.sum(slit_mask * lya_convolved)
            fluxes_convolved.append(flux_convolved)

            # Calculate the uncertainty of the convolved flux
            n_flux_convolved = np.sqrt(np.sum((slit_mask * n_lya)**2))
            n_fluxes_convolved.append(n_flux_convolved)

            # Calculate the ratio between the convolved and unconvolved flux
            ratio = flux_convolved / flux
            ratios.append(ratio)

            # Calculate the propagated uncertainty of the ratio between the convolved and unconvolved Lya flux
            n_ratio = np.absolute(ratio) * np.sqrt((n_flux / flux)**2 + (n_flux_convolved / flux_convolved)**2)
            n_ratios.append(n_ratio)

    header = 'The Lyα fluxes and ratios measured in the seeing effects simulation\n' \
        + f'Created by seeing.ipynb on {datetime.now(timezone.utc).isoformat().replace("+00:00", "Z")}\n' \
        + '\n' \
        + 'The columns below are organized as follows. There are three types of measurements: unconvolved and convolved Lyα fluxes in a slit aperture (e/s),\n' \
        + 'and the ratio between the two fluxes. Each measurement type has a set of 4 columns, arranged consecutively in the order mentioned previously.\n' \
        + 'Those sets of 4 columns each have 2 consecutive subsets of 2 columns. In each subset, the first two columns represent, respectively, the\n' \
        + 'measurement and uncertainty from the F390W-based Lyα map, and the last two columns represent, respectively, the measurement and uncertainty\n' \
        + 'from the F555W-based Lyα map. The rows corresponding to the different MagE slit apertures are arranged, from top to bottom, in the following\n' \
        + 'order: M5, M4, M6, M3, M0, M2, M7, M8, and M9.\n' \
    
    np.savetxt(f'{results}/seeing_simulation_measurements_results.txt', 
        np.array([fluxes[::2], n_fluxes[::2], fluxes[1::2], n_fluxes[1::2], fluxes_convolved[::2], n_fluxes_convolved[::2], fluxes_convolved[1::2], n_fluxes_convolved[1::2], ratios[::2], n_ratios[::2], ratios[1::2], n_ratios[1::2]], dtype=np.float64).T, 
        delimiter=' ', header=header, encoding='utf-8')

def tabulate():

    def round_to_uncertainties(measurement, uncertainty):

        '''
        Round the uncertainty to one significant figure and the measurement to
        the same significant digit as the uncertainty

        Parameters:

            measurement : numpy.float64
                Value of the measurement

            uncertainty : numpy.float64
                Estimated uncertainty of the measurement

        Returns:

            measurement : str
                Value of the measurement, rounded to the same significant digit as
                the 1 significant figure-rounded uncertainty

            uncertainty : str
                Estimated uncertainty of the measurement, rounded to 1 significant figure
        '''

        # Round the uncertainty to one significant figure
        uncertainty = sigfig.round(uncertainty, sigfigs=1, type=str)

        # If the uncertainty is less than 1
        if '.' in uncertainty:

            # Round the measurement to the same digit as the uncertainty
            measurement = sigfig.round(measurement, decimals=len(uncertainty.split('.')[1]), type=str)

        # If the uncertainty is greater than 1
        elif '.' not in uncertainty:

            # Round the measurement to the same digit as the uncertainty
            measurement = sigfig.round(measurement, len(str(measurement).replace('-','').split('.')[0]) - len(uncertainty) + 1, type=str)

        return measurement, uncertainty

    # Dictionary of slit IDs
    slits = {
        'M5' : [],
        'M4' : [],
        'M6' : [],
        'M3' : [],
        'M0' : [],
        'M2' : [],
        'M7' : [],
        'M8' : [],
        'M9' : []
    }

    # Establish common directories
    home = os.getcwd()
    results = f'{home}/results'

    # Make the header of the seeing simulation results table
    table = '\\begin{deluxetable*}{r|lll|lll}\n' \
        + '\n' \
        + '\\tablecaption{Simulated aperture flux changes due to atmospheric seeing \label{tab:seeing_simulation}}\n' \
        + '\n' \
        + '\\tablehead{\n' \
        + '\t\colhead{Slit} & \n' \
        + '\t\colhead{} &\n' \
        + '\t\colhead{F390W} &\n' \
        + '\t\colhead{} &\n' \
        + '\t\colhead{} &\n' \
        + '\t\colhead{F555W} & \n' \
        + '\t\colhead{}\n' \
        + '\t\\\\\n' \
        + '\t\colhead{} &\n' \
        + '\t\colhead{Unconvolved} &\n' \
        + '\t\colhead{Convolved} &\n' \
        + '\t\colhead{Ratio} & \n' \
        + '\t\colhead{Unconvolved} &\n' \
        + '\t\colhead{Convolved} &\n' \
        + '\t\colhead{Ratio}\n' \
        + '\t\\\\\n' \
        + '\t\colhead{} &\n' \
        + '\t\colhead{[$e^-$ s$^{-1}$]} &\n' \
        + '\t\colhead{[$e^-$ s$^{-1}$]} &\n' \
        + '\t\colhead{} &\n' \
        + '\t\colhead{[$e^-$ s$^{-1}$]} &\n' \
        + '\t\colhead{[$e^-$ s$^{-1}$]} &\n' \
        + '\t\colhead{}\n' \
        + '}\n' \
        + '\n' \
        + '\startdata\n'

    # Get the saved results of the seeing simulation
    lya_measurements = np.loadtxt(f'{results}/seeing_simulation_measurements_results.txt', delimiter=' ', comments='#')

    # For each slit ID
    for i, slit_id in enumerate(slits):

        # Get the row of measurements from the seeing simulation corresponding to the slit aperture
        slit_lya_measurements = lya_measurements[i]

        # Round the table quantities to the appropriate number of significant figures
        f390w_flux, n_f390w_flux = round_to_uncertainties(slit_lya_measurements[0], slit_lya_measurements[1])
        f390w_flux_convolved, n_f390w_flux_convolved = round_to_uncertainties(slit_lya_measurements[4], slit_lya_measurements[5])
        f390w_ratio, n_f390w_ratio = round_to_uncertainties(slit_lya_measurements[8], slit_lya_measurements[9])

        f555w_flux, n_f555w_flux = round_to_uncertainties(slit_lya_measurements[2], slit_lya_measurements[3])
        f555w_flux_convolved, n_f555w_flux_convolved = round_to_uncertainties(slit_lya_measurements[6], slit_lya_measurements[7])
        f555w_ratio, n_f555w_ratio = round_to_uncertainties(slit_lya_measurements[10], slit_lya_measurements[11])

        # Add a separating horizontal line between the LyC-leaking and non-LyC-leaking spectra
        table = table + '\hline\n' if slit_id == 'M0' else table + ''

        # Format the table row for the slit
        table = table + f'{slit_id} & ${f390w_flux}\pm{n_f390w_flux}$ & ${f390w_flux_convolved}\pm{n_f390w_flux_convolved}$ & ${f390w_ratio}\pm{n_f390w_ratio}$ ' \
            + f'& ${f555w_flux}\pm{n_f555w_flux}$ & ${f555w_flux_convolved}\pm{n_f555w_flux_convolved}$ & ${f555w_ratio}\pm{n_f555w_ratio}$'

        # Add a LaTeX row break character and start a new line, unless the row is the last row, then just add a new line
        table = table + ' \\\\\n' if i != len(slits) - 1 else table + '\n'

    # Add the table footer
    table = table + '\enddata\n' \
        + '\n' \
        + '\\tablecomments{From left to right: slit, flux inside the slit of the unconvolved and simulated seeing-convolved F390W-based Ly$\\alpha$ map, the ratio between the two fluxes, and likewise for the F555W-based Ly$\\alpha$ map. Ratios $>$ 1 indicate the flux in the aperture increased after the convolution, and ratios $<$ 1 indicate the flux in the aperture decreased after the convolution. The convolution used a 2-dimensional Gaussian kernel of the combined effect of the time-weighted seeing conditions and airmasses (\S\,\\ref{sssec:disc_lya_seeing}).}\n' \
        + '\n' \
        + '\end{deluxetable*}'

    # Save the table
    f = open(f'{results}/tables/seeing_simulation_measurements_table.tex', 'w', encoding='utf-8')
    f.write(table)
    f.close()

In [25]:
func()

In [26]:
tabulate()